In [1]:
from scipy.stats import skewnorm
import torch

In [5]:
# 1. marginKLDiv的negative影响大
a = 4
mean, var, skew, kurt = skewnorm.stats(a, moments='mvsk')

scores = skewnorm.rvs(a, size=10)

In [23]:
scores = [3, 2.5, 1, 2, 1.2]
labels = torch.log_softmax(torch.tensor([5.0,4,3,2,1]),0)

In [24]:
ls_scores = torch.log_softmax(torch.tensor(scores),0)

In [25]:
loss_fct = torch.nn.KLDivLoss(reduction='none', log_target=True)

In [27]:
loss_fct(ls_scores, labels)

tensor([ 0.2355, -0.0304,  0.0319, -0.0516, -0.0213])

In [6]:
marginal_scores1 = [1, 2.0]
marginal_scores2 = [x - 2 for x in marginal_scores1]
labels = [0, 2.5, 3, 2]

def ls(l):
    return torch.log_softmax(torch.tensor(l), 0)

def s(l):
    return torch.nn.Softmax()(torch.tensor(l))



In [68]:
print(s([-1.5,2,3,4]))
#假设postive分数是0
print(s([1.5,-2,-3,-4]))

tensor([0.0027, 0.0898, 0.2441, 0.6634])
tensor([0.9566, 0.0289, 0.0106, 0.0039])


/tmp/ipykernel_17478/1839917421.py:9: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return torch.nn.Softmax()(torch.tensor(l))


In [69]:
marginal_scores1 = [1, 4.0, 10, 39]
label1 = [2.5, 3, 3.5, 23]

In [70]:
loss_fct(ls(marginal_scores1), ls(label1))

tensor([2.1878e-08, 3.0917e-08, 3.2284e-08, 0.0000e+00])

In [71]:
loss_fct(ls([-x for x in marginal_scores1]), ls([-x for x in label1]))

tensor([-3.1988e-01,  5.7398e-01,  1.3729e+00,  1.0681e-08])

In [24]:
model_dir = "training_with_sentence_transformers/output/colbert_splade_distill_num1_origneg_marginkldiv_position_mrr_diff_alpha0.0_denoiseFalse_num20_marginkldiv_position5-lr1e-06-batch_size_8x4-2022-10-18/100000/0_ColBERTTransformer"


In [25]:
import torch
import os
from training_with_sentence_transformers.models import MLMTransformerDense, ColBERTTransformer
model = ColBERTTransformer(model_dir, max_seq_length=256)
checkpoint = torch.load(os.path.join(model_dir, "checkpoint.pt"), map_location='cpu')
model.load_state_dict(checkpoint)

<All keys matched successfully>

In [15]:
from collections import defaultdict
run = defaultdict(dict)
with open("../msmarco/queries.train.index_splade_num1_marginmse.psg.train.trec") as f:
    for line in f:
        qid, _, did, _, score, _ = line.strip().split("\t")
        run[qid][did] = float(score)

In [ ]:
import os
import gzip
import pickle
ce_scores_file = os.path.join('../msmarco/cross-encoder-ms-marco-MiniLM-L-6-v2-scores.pkl.gz')
with gzip.open(ce_scores_file, 'rb') as fIn:
    ce_scores = pickle.load(fIn)

run_clean = defaultdict(dict)
run_ce = defaultdict(dict)
for q in run:
    for d in run[q]:
        if d in ce_scores:
            run_ce[q][d] = ce_scores[q][d]
            run_clean[q][d] = run[q][d]
            

In [2]:
qrel_file = f"../msmarco/qrels.dev.tsv" 
qrels = defaultdict(dict)
with open(qrel_file) as f:
    for line in f:
        try:
            qid, _, did, rel = line.strip().split("\t")
        except:
            qid, _, did, rel = line.strip().split(" ")
        if int(rel) > 0:
            qrels[qid][did] = int(rel)

In [4]:
import pytrec_eval
trec_eval = pytrec_eval.RelevanceEvaluator(qrels, {'recip_rank'})
eval_scores = trec_eval.evaluate(run)

In [5]:
for q in run:
    ls(run[q])

{'188714': {'recip_rank': 1.0},
 '1082792': {'recip_rank': 1.0},
 '995526': {'recip_rank': 1.0},
 '199776': {'recip_rank': 0.023255813953488372},
 '660957': {'recip_rank': 0.1},
 '820267': {'recip_rank': 0.1},
 '837202': {'recip_rank': 0.020833333333333332},
 '130825': {'recip_rank': 0.25},
 '408149': {'recip_rank': 0.2},
 '345453': {'recip_rank': 0.14285714285714285},
 '1019649': {'recip_rank': 0.0},
 '1099065': {'recip_rank': 0.041666666666666664},
 '542431': {'recip_rank': 0.03225806451612903},
 '1084910': {'recip_rank': 1.0},
 '959083': {'recip_rank': 0.14285714285714285},
 '816483': {'recip_rank': 0.027777777777777776},
 '995825': {'recip_rank': 1.0},
 '559771': {'recip_rank': 0.16666666666666666},
 '1091246': {'recip_rank': 0.5},
 '167436': {'recip_rank': 0.5},
 '899212': {'recip_rank': 0.16666666666666666},
 '1101211': {'recip_rank': 1.0},
 '1047854': {'recip_rank': 0.05},
 '991832': {'recip_rank': 0.3333333333333333},
 '426442': {'recip_rank': 1.0},
 '1093443': {'recip_rank': 1